In [1]:
import requests
import pandas as pd

## Pobieranie wyników turnieju- część 1

In [2]:
def scores_pipeline(url):
    pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
    df=pd.read_csv('mydata.csv')
    df=df.drop(['Unnamed: 0','+/-','pkl', 'pdf'],axis=1)
    return df

In [3]:
def regex_trans_scores(df):
    import re
    pierwszy=[]
    drugi=[]
    for i in range(len(df)):
        pierwszy.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[0]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[1])
        drugi.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[2]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[3])
    df['nazwisko_1']=pierwszy
    df['nazwisko_2']=drugi
    df=df.drop('nazwiska',axis=1)


    wk1=[]
    wk2=[]
    for i in range(len(df)):
        wk1.append(re.findall("\d+\.\d", df.wk[i])[0])
        wk2.append(re.findall("\d+\.\d", df.wk[i])[1])
    df['wk_1']=wk1
    df['wk_2']=wk2
    df=df.drop(['wk','ośrodek','okręg'],axis=1)
    return df

## Pobieranie wyników poszczególnych rozdań

In [110]:
def get_hist(n,vector):
    pd.read_html(requests.get(f"https://r.bridgespider.com/{vector[0]}/prot/{1}/").content)[-1].to_csv("mydat.csv")
    tmp=pd.read_csv('mydat.csv')
    tmp['nr_rozdania']=1
    tmp['nr_turnieju']=vector[0]
    for j in range (len(vector)):
        for i in range(1,n):
            pd.read_html(requests.get(f"https://r.bridgespider.com/{vector[j]}/prot/{i+1}/").content)[-1].to_csv("mydat.csv")
            zz=pd.read_csv('mydat.csv')
            zz['nr_rozdania']=i+1
            zz['nr_turnieju']=vector[j]
            tmp=pd.concat([tmp,zz])
    return tmp

In [104]:
tmp=pd.read_html(requests.get(f"https://r.bridgespider.com/{25890}/prot/{1}/").content)[-1].to_csv("mydat.csv")

In [107]:
tmp=pd.read_csv('mydat.csv')

In [109]:
pd.concat([tmp,tmp])

,Unnamed: 0,ośrodek,runda,NS,EW,kontrakt,rozg.,wist,lew,zapis,% NS,% EW,Unnamed: 11
0,0,DS/136,5,8,11,2,E,8,+1,-150,67.46,32.54,NaN
1,1,DS/136,3,5,9,1,W,5,+3,-180,57.74,42.26,NaN
2,2,DS/136,6,10,7,2,E,3,+2,-180,57.74,42.26,NaN
3,3,DS/136,4,3,2,3,E,3,=,-400,38.89,61.11,NaN
4,4,DS/136,9,4,6,3,E,8,=,-400,38.89,61.11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,248,ZP/452,11,530,532,3,E,3,-2,100,98.61,1.39,NaN
249,249,ZP/452,12,525,528,3,E,6,=,-400,38.89,61.11,NaN
250,250,ZP/452,13,527,523,3,W,5,=,-400,38.89,61.11,NaN
251,251,ZP/452,14,529,522,3,W,5,+1,-430,13.29,86.71,NaN


In [5]:
def pipeline(df2):
    df2=df2.drop(['Unnamed: 0','ośrodek','runda','wist','Unnamed: 11','lew','kontrakt'],axis=1)
    df2['rozg.']=df2['rozg.'].replace(['E','W'],'EW')
    df2['rozg.']=df2['rozg.'].replace(['N','S'],'NS')
    df2['zapis_NS']=df2['zapis']
    df2['zapis_EW']=df2['zapis']*(-1)
    df2=df2.drop('zapis',axis=1)
    return df2

In [72]:
def pipeline2(df_x):
    tmp1=df_x.copy()
    tmp2=df_x.copy()
    tmp1=tmp1.drop(['EW','% EW','zapis_EW'],axis=1)
    tmp1['linia']='NS'
    tmp2=tmp2.drop(['NS','% NS','zapis_NS'],axis=1)
    tmp2['linia']='EW'
    tmp1.columns=['id','rozg','wynik','nr_rozdania','nr_turnieju','zapis','linia']
    tmp2.columns=['id','rozg','wynik','nr_rozdania','nr_turnieju','zapis','linia']
    tmp=pd.concat([tmp1,tmp2])
    return tmp

In [116]:
def refactor(history):
    history=pipeline(history)
    history=pipeline2(history)
    return history

## Main

In [8]:
scores=scores_pipeline("https://r.bridgespider.com/25890/")
scores=regex_trans_scores(scores)
scores['nr_turnieju']=25890
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,99,66.18,Jerzy Pala,Jan Styczyński,5.0,5.0,25890
1,2,140,65.97,Sebastian Rawlik,Damian Mazurak,5.0,5.0,25890
2,3,10,64.60,Janusz Czerniawski,Zbigniew Buciuto,5.0,3.0,25890
3,4,527,64.47,Ryszard Smejda,Damian Wroński,7.0,11.0,25890
4,5,247,64.31,Wojciech Słysz,Marek Melsztyński,1.5,2.0,25890
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


In [9]:
history=get_hist(30)
history=refactor(history)
history[0]

,id,rozg,wynik,zapis,linia
0,8,EW,67.46,-150,NS
1,5,EW,57.74,-180,NS
2,10,EW,57.74,-180,NS
3,3,EW,38.89,-400,NS
4,4,EW,38.89,-400,NS
...,...,...,...,...,...
248,532,EW,1.39,-100,EW
249,528,EW,61.11,400,EW
250,523,EW,61.11,400,EW
251,522,EW,86.71,430,EW


## Część 2

In [10]:
for i in range(30):
    history[i]['numer_rozdania']=i+1    

In [11]:
tmp=pd.concat([history[0],history[1]])
for i in range(2,len(history)):
    tmp=pd.concat([tmp,history[i]])
tmp    

,id,rozg,wynik,zapis,linia,numer_rozdania
0,8,EW,67.46,-150.0,NS,1
1,5,EW,57.74,-180.0,NS,1
2,10,EW,57.74,-180.0,NS,1
3,3,EW,38.89,-400.0,NS,1
4,4,EW,38.89,-400.0,NS,1
...,...,...,...,...,...,...
136,531,EW,78.84,170.0,EW,30
137,527,EW,78.84,170.0,EW,30
138,522,EW,78.84,170.0,EW,30
139,521,EW,56.41,140.0,EW,30


In [12]:
tmp1=tmp[tmp['rozg']==tmp['linia']]
tmp2=tmp[tmp['rozg']!=tmp['linia']]
tmp1['czy_rozgrywa']=True
tmp2['czy_rozgrywa']=False
tmp=pd.concat([tmp1,tmp2])

C:\Users\p.okonek\AppData\Local\Temp\ipykernel_15768\3798423981.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['czy_rozgrywa']=True
C:\Users\p.okonek\AppData\Local\Temp\ipykernel_15768\3798423981.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2['czy_rozgrywa']=False


In [13]:
tmp[tmp['id']==99].reset_index()

,index,id,rozg,wynik,zapis,linia,numer_rozdania,czy_rozgrywa
0,47,99,EW,61.11,400.0,EW,1,True
1,47,99,EW,89.00,1010.0,EW,2,True
2,51,99,NS,69.48,650.0,NS,4,True
3,51,99,NS,58.53,140.0,NS,5,True
4,51,99,NS,82.74,460.0,NS,6,True
5,51,99,EW,98.40,590.0,EW,8,True
6,48,99,NS,54.96,140.0,NS,11,True
7,52,99,EW,96.63,170.0,EW,16,True
8,52,99,EW,88.25,-100.0,EW,18,True
9,50,99,NS,24.70,180.0,NS,21,True


In [14]:
tab=[25845,25809,25773,25737,25692,24384,24339,24303,24267,24222,24186,24150,22079 ,22043 ,21800,21764,21728,17693,17666]

In [15]:
url="https://r.bridgespider.com/"

In [16]:
url+str(tab[2])+'/'

'https://r.bridgespider.com/25773/'

In [17]:
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,99,66.18,Jerzy Pala,Jan Styczyński,5.0,5.0,25890
1,2,140,65.97,Sebastian Rawlik,Damian Mazurak,5.0,5.0,25890
2,3,10,64.60,Janusz Czerniawski,Zbigniew Buciuto,5.0,3.0,25890
3,4,527,64.47,Ryszard Smejda,Damian Wroński,7.0,11.0,25890
4,5,247,64.31,Wojciech Słysz,Marek Melsztyński,1.5,2.0,25890
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


-------------------------------

In [18]:
for i in tab:
    url="https://r.bridgespider.com/"
    url=url+str(i)+'/'
    tmp=scores_pipeline(url)
    tmp=regex_trans_scores(tmp)
    tmp['nr_turnieju']=i
    scores=pd.concat([tmp,scores])
    
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


In [19]:
scores['unique_id'] = scores.groupby(['nazwisko_1', 'nazwisko_2']).ngroup()
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666,1199
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666,2029
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666,883
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666,4565
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666,1865
...,...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890,2078
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890,2591
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890,3234
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890,4430


In [20]:
vector=scores["unique_id"].value_counts().index.tolist()[:10]

In [21]:
top10=scores.loc[scores["unique_id"].isin(vector)]

In [22]:
top10

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
55,56,700,59.53,Rafał Junik,Jolanta Ziętara,7.0,5.0,17666,3473
148,149,592,55.67,Wiesław Pietrzak,Zenon Wawer,5.0,5.0,17666,4382
237,238,207,52.96,Krzysztof Borowik,Henryk Choiński,0.0,3.0,17666,2359
250,251,202,52.59,Franciszek Bułyszko,Zbigniew Józefiak,4.0,4.0,17666,1158
318,319,13,51.25,Andrzej Augustyniak,Henryk Gayny,5.0,2.0,17666,178
...,...,...,...,...,...,...,...,...,...
193,194,388,52.54,Wiesław Pietrzak,Zenon Wawer,5.0,5.0,25890,4382
285,286,198,49.09,Krzysztof Borowik,Henryk Choiński,1.5,3.0,25890,2359
405,406,200,44.91,Sławomir Rutecki,Tadeusz Birecki,7.0,5.0,25890,3963
441,442,199,43.00,Franciszek Bułyszko,Zbigniew Józefiak,4.0,5.0,25890,1158


## Zawodnicy z Cezara

In [23]:
url="https://msc.com.pl/cezar/?p=51&rok=0&page=1"

In [24]:
pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
df=pd.read_csv('mydata.csv')

In [25]:
tables = pd.read_html(url,match="klub / drużyna")
df=tables[2]

In [26]:
for i in range(65,337):
    url=f"https://msc.com.pl/cezar/?p=51&rok=0&page={i}"
    tables = pd.read_html(url,match="klub / drużyna")
    df=pd.concat([df,tables[2]])

In [27]:
df.to_csv('Lista_Zawodnikow.csv')

In [28]:
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666,1199
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666,2029
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666,883
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666,4565
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666,1865
...,...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890,2078
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890,2591
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890,3234
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890,4430


In [29]:
scores.columns

Index(['m-ce', 'nr', '%', 'nazwisko_1', 'nazwisko_2', 'wk_1', 'wk_2',
       'nr_turnieju', 'unique_id'],
      dtype='object')

In [30]:
scores_1=scores[['m-ce', 'nr', '%', 'nazwisko_1','wk_1','nr_turnieju']]
scores_2=scores[['m-ce', 'nr', '%', 'nazwisko_2','wk_2','nr_turnieju']]

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
scores_1['nazwisko']=scores_1['nazwisko_1']
scores_2['nazwisko']=scores_2['nazwisko_2']
scores_1['wk']=scores_1['wk_1']
scores_2['wk']=scores_2['wk_2']
scores_1=scores_1.drop(['nazwisko_1','wk_1'],axis=1)
scores_2=scores_2.drop(['nazwisko_2','wk_2'],axis=1)

In [33]:
scores_new=pd.concat([scores_1,scores_2])

In [34]:
df.rename(columns = {2: 'name'}, inplace = True)

In [35]:
df.rename(columns = {1: 'PID'}, inplace = True)

In [36]:
df

,0,PID,name,3,4,5,6,7,8
0,lokata,PID,imię i nazwisko,WZBS,WK,TK,PKL,liga,klub / drużyna
1,1.,06706,Apolinary Kowalski,WM,24,AS,187 649,E,VITAS ESBS Elblag
2,2.,08513,Piotr Gawryś,DS,24,AS,149 944,E,BRIDGESCANNER KS AZS Wratislavia I WR
3,3.,06658,Jacek Kalita,SW,24,AS,146 928,E,SPS CONSTRUCTION Kielce
4,4.,01221,Cezary Balicki,DS,24,AS,143 379,-,NaN
...,...,...,...,...,...,...,...,...,...
46,NaN,20956,Maciej Wisz,PK,0,D,1,-,Singleton Podkarpacki
47,NaN,17464,Małgorzata Wiśniewska,DS,0,D,1,-,NaN
48,NaN,20096,Adrian Witkowski,KP,0,D,1,-,NaN
49,NaN,21716,Jakub Witowski,PK,0,D,1,-,Singleton Podkarpacki


In [121]:
last_scores=scores_new.merge(df[['PID','name']],left_on='nazwisko',right_on='name')

In [122]:
last_scores=last_scores.drop(['name'],axis=1)

In [123]:
last_scores

,m-ce,nr,%,nr_turnieju,nazwisko,wk,PID
0,3,374,65.35,17666,Dariusz Krakowiak,3.0,17854
1,33,384,61.43,17693,Dariusz Krakowiak,3.0,17854
2,183,366,55.17,21728,Dariusz Krakowiak,3.0,17854
3,49,384,60.59,21764,Dariusz Krakowiak,3.0,17854
4,72,52,46.54,24150,Dariusz Krakowiak,3.0,17854
...,...,...,...,...,...,...,...
11754,232,176,51.10,25890,Andrzej Piętak,3.0,08323
11755,303,455,48.75,25890,Andrzej Grześko,2.0,19403
11756,321,20,48.22,25890,Tymoteusz Zaborowski,1.5,17787
11757,467,411,40.96,25890,Wiesław Kulikowski,1.5,15109


In [134]:
zz=last_scores[last_scores[['nr_turnieju','nazwisko']].duplicated(keep=False)]

In [135]:
last_scores=pd.concat([last_scores,zz]).drop_duplicates(keep=False)

In [44]:
tmp

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666
...,...,...,...,...,...,...,...,...
747,748,459,34.16,Adam Maryniak,Bożena Materna,0.0,0.0,17666
748,749,66,33.28,Jerzy Stypiński,Tadeusz Janeczek,3.0,3.0,17666
749,750,431,33.07,Ireneusz Hebdowski,Jan Kurianowicz,0.0,0.0,17666
750,751,5,31.37,Zbigniew Leśniewski,Mikołaj Piotrowski,1.5,0.0,17666


In [137]:
last_scores

,m-ce,nr,%,nr_turnieju,nazwisko,wk,PID
0,3,374,65.35,17666,Dariusz Krakowiak,3.0,17854
1,33,384,61.43,17693,Dariusz Krakowiak,3.0,17854
2,183,366,55.17,21728,Dariusz Krakowiak,3.0,17854
3,49,384,60.59,21764,Dariusz Krakowiak,3.0,17854
4,72,52,46.54,24150,Dariusz Krakowiak,3.0,17854
...,...,...,...,...,...,...,...
11754,232,176,51.10,25890,Andrzej Piętak,3.0,08323
11755,303,455,48.75,25890,Andrzej Grześko,2.0,19403
11756,321,20,48.22,25890,Tymoteusz Zaborowski,1.5,17787
11757,467,411,40.96,25890,Wiesław Kulikowski,1.5,15109


In [136]:
last_scores['nazwisko'].value_counts()

Dariusz Krakowiak      20
Zbigniew Bobak         20
Przemysław Wojtczak    20
Henryk Choiński        19
Krzysztof Borowik      19
                       ..
Antoni Kierepka         1
Marek Księżopolski      1
Szymon Mazur            1
Janusz Kurkowski        1
Matylda Brzyzka         1
Name: nazwisko, Length: 2003, dtype: int64

## Pobieranie wynikow konkretnego gracza

In [47]:
vector=last_scores[last_scores['nazwisko']=='Dariusz Krakowiak']['nr_turnieju']

In [54]:
vector

0     17666
1     17693
2     21728
3     21764
4     24150
5     24267
6     24303
7     25845
8     21800
9     22043
10    22079
11    24186
12    24222
13    24339
14    24384
15    25692
16    25737
17    25773
18    25809
19    25890
Name: nr_turnieju, dtype: int64

In [57]:
history[29]

,id,rozg,wynik,zapis,linia,numer_rozdania
0,9,NS,90.94,110,NS,30
1,3,EW,43.59,-140,NS,30
2,2,EW,21.16,-170,NS,30
3,4,EW,1.94,-420,NS,30
4,7,EW,1.94,-420,NS,30
...,...,...,...,...,...,...
136,531,EW,78.84,170,EW,30
137,527,EW,78.84,170,EW,30
138,522,EW,78.84,170,EW,30
139,521,EW,56.41,140,EW,30


In [58]:
proba=[25890,25809]

In [62]:
history

[     Unnamed: 0 ośrodek  runda   NS   EW kontrakt rozg. wist lew  zapis  \
 0             0  DS/136      1    3    7        3     N    2   =    140   
 1             1  DS/136      7    5    6        3     N    9  -1    -50   
 2             2  DS/136      8   10    8        3     S    3  -3   -150   
 3             3  DS/136      9    1    4      3 x     N    9  -2   -300   
 4             4  DS/136      6    2    9        3     W    J   =   -400   
 ..          ...     ...    ...  ...  ...      ...   ...  ...  ..    ...   
 267         267  ZP/433      5  586  593        2     S    4  -1    -50   
 268         268  ZP/433      6  592  590        3     W    4  +1   -430   
 269         269  ZP/433      7  594  587        3     W    9  -1     50   
 270         270  ZP/433      8  585  591        3     E    J  +2   -150   
 271         271  ZP/433      9  584  595        2     N    2   =    110   
 
       % NS   % EW  Unnamed: 11  
 0    94.65   5.35          NaN  
 1    52.40  47.60

In [111]:
history=get_hist(30,proba)   

In [115]:
history=pipeline(history)
history=pipeline2(history)
history

,id,rozg,wynik,nr_rozdania,nr_turnieju,zapis,linia
0,8,EW,67.46,1,25890,-150.0,NS
1,5,EW,57.74,1,25890,-180.0,NS
2,10,EW,57.74,1,25890,-180.0,NS
3,3,EW,38.89,1,25890,-400.0,NS
4,4,EW,38.89,1,25890,-400.0,NS
...,...,...,...,...,...,...,...
197,589,NS,44.76,30,25809,-420.0,EW
198,586,NS,70.47,30,25809,-170.0,EW
199,590,NS,16.79,30,25809,-450.0,EW
200,595,NS,52.50,30,25809,-400.0,EW


In [113]:
len(history)

15003

In [96]:
tmp=pd.DataFrame(history[0])

In [97]:
tmp

,Unnamed: 0,ośrodek,runda,NS,EW,kontrakt,rozg.,wist,lew,zapis,% NS,% EW,Unnamed: 11,nr_rozdania,nr_turnieju
0,0,DS/136,5,8,11,2,E,8,+1,-150,67.46,32.54,NaN,1,25890
1,1,DS/136,3,5,9,1,W,5,+3,-180,57.74,42.26,NaN,1,25890
2,2,DS/136,6,10,7,2,E,3,+2,-180,57.74,42.26,NaN,1,25890
3,3,DS/136,4,3,2,3,E,3,=,-400,38.89,61.11,NaN,1,25890
4,4,DS/136,9,4,6,3,E,8,=,-400,38.89,61.11,NaN,1,25890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,248,ZP/452,11,530,532,3,E,3,-2,100,98.61,1.39,NaN,1,25890
249,249,ZP/452,12,525,528,3,E,6,=,-400,38.89,61.11,NaN,1,25890
250,250,ZP/452,13,527,523,3,W,5,=,-400,38.89,61.11,NaN,1,25890
251,251,ZP/452,14,529,522,3,W,5,+1,-430,13.29,86.71,NaN,1,25890


### TODO
numer borda, numer turnieju, konkatencja

## Pobieranie wyników z poszczególnych turniejów

In [144]:
proba=[25845,25809,25773,25737,25692,24384,24339,24303,24267,24222,24186,24150,22079 ,22043 ,21800,21764,21728,17693,17666]
history=get_hist(30,proba)   
history=pipeline(history)
history=pipeline2(history)
history

,id,rozg,wynik,nr_rozdania,nr_turnieju,zapis,linia
0,4,EW,99.21,1,25845,500.0,NS
1,6,NS,93.68,1,25845,460.0,NS
2,10,NS,88.93,1,25845,450.0,NS
3,5,NS,82.02,1,25845,430.0,NS
4,2,NS,40.12,1,25845,140.0,NS
...,...,...,...,...,...,...,...
347,780,EW,84.00,30,17666,990.0,EW
348,776,EW,84.00,30,17666,990.0,EW
349,772,EW,84.00,30,17666,990.0,EW
350,778,EW,53.29,30,17666,980.0,EW


In [124]:
last_scores

,m-ce,nr,%,nr_turnieju,nazwisko,wk,PID
0,3,374,65.35,17666,Dariusz Krakowiak,3.0,17854
1,33,384,61.43,17693,Dariusz Krakowiak,3.0,17854
2,183,366,55.17,21728,Dariusz Krakowiak,3.0,17854
3,49,384,60.59,21764,Dariusz Krakowiak,3.0,17854
4,72,52,46.54,24150,Dariusz Krakowiak,3.0,17854
...,...,...,...,...,...,...,...
11754,232,176,51.10,25890,Andrzej Piętak,3.0,08323
11755,303,455,48.75,25890,Andrzej Grześko,2.0,19403
11756,321,20,48.22,25890,Tymoteusz Zaborowski,1.5,17787
11757,467,411,40.96,25890,Wiesław Kulikowski,1.5,15109


In [145]:
tmp=history.merge(last_scores, left_on=['id','nr_turnieju'],right_on=['nr','nr_turnieju'])

In [147]:
tmp

,id,rozg,wynik,nr_rozdania,nr_turnieju,zapis,linia,m-ce,nr,%,nazwisko,wk,PID
0,4,EW,99.21,1,25845,500.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482
1,4,EW,99.21,1,25845,500.0,NS,371,4,46.28,Zbigniew Bobak,4.0,01247
2,4,EW,14.82,2,25845,-140.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482
3,4,EW,14.82,2,25845,-140.0,NS,371,4,46.28,Zbigniew Bobak,4.0,01247
4,4,NS,26.77,3,25845,170.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274626,382,NS,50.57,28,17666,-140.0,EW,142,382,56.07,Michał Szadkowski,0.0,18841
274627,382,NS,97.01,29,17666,100.0,EW,142,382,56.07,Wojciech Wojsa,0.5,13918
274628,382,NS,97.01,29,17666,100.0,EW,142,382,56.07,Michał Szadkowski,0.0,18841
274629,382,EW,27.43,30,17666,490.0,EW,142,382,56.07,Wojciech Wojsa,0.5,13918


In [214]:
tmp.to_csv("proba.csv")

In [146]:
tmp['nazwisko'].value_counts()

Dariusz Krakowiak       530
Zbigniew Bobak          512
Przemysław Wojtczak     511
Dominik Głowacki        502
Henryk Choiński         498
                       ... 
Grzegorz Wasiak          23
Jacek Węgrzyk            23
Norbert Krzystanek       23
Andrzej Górniakowski     23
Robert Mroczek           23
Name: nazwisko, Length: 1985, dtype: int64

## Podział rozgrywka, obrona

In [153]:
zz=tmp[tmp['rozg']==tmp['linia']]
zz2=tmp[tmp['rozg']!=tmp['linia']]
zz['czy_rozgrywa']=True
zz2['czy_rozgrywa']=False
tmp=pd.concat([zz,zz2])

In [159]:
import math

In [199]:
tmp1=tmp[ (tmp['zapis']>0) & (tmp['czy_rozgrywa'] )]
tmp1['zapis2']=pd.cut(tmp['zapis'], bins=[0, 300, 900, math.inf],labels=['częściówka','końcówka','szlemik'], include_lowest=True)

In [200]:
tmp2=tmp[ (tmp['zapis']<0) & (tmp['czy_rozgrywa']==False )]
tmp2['zapis2']=pd.cut(tmp['zapis'], bins=[-math.inf,-900,-300,0],labels=['szlemik','końcówka','częściówka'], include_lowest=True)

In [201]:
tmp3=tmp[ (tmp['zapis']>=0) & (tmp['czy_rozgrywa']==False )]
tmp3['zapis2']='Kokosy_na_Obronie'

In [202]:
tmp4=tmp[ (tmp['zapis']<=0) & (tmp['czy_rozgrywa']==True )]
tmp4['zapis2']='Wpadka'

In [203]:
tmp=pd.concat([tmp1,tmp2,tmp3,tmp4])

## Statystyki posczególnych zawodników

### Średnie

In [321]:
stats2=tmp[tmp['czy_rozgrywa']==True].groupby(['PID'])['wynik'].mean().reset_index(name='średnia_na_rozgrywce')

In [322]:
stats3=tmp[tmp['czy_rozgrywa']==False].groupby(['PID'])['wynik'].mean().reset_index(name='średnia_na_obronie')

In [323]:
stat=stats2.merge(stats3,on='PID')

In [326]:
tmp

,id,rozg,wynik,nr_rozdania,nr_turnieju,zapis,linia,m-ce,nr,%,nazwisko,wk,PID,czy_rozgrywa,zapis2
4,4,NS,26.77,3,25845,170.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482,True,częściówka
5,4,NS,26.77,3,25845,170.0,NS,371,4,46.28,Zbigniew Bobak,4.0,01247,True,częściówka
8,4,NS,63.39,8,25845,400.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482,True,końcówka
9,4,NS,63.39,8,25845,400.0,NS,371,4,46.28,Zbigniew Bobak,4.0,01247,True,końcówka
16,4,NS,51.98,18,25845,130.0,NS,371,4,46.28,Stefan Małowiecki,4.0,01482,True,częściówka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274570,244,EW,35.59,27,17666,-100.0,EW,286,244,51.96,Jerzy Jakubowski,1.5,18390,True,Wpadka
274597,382,EW,59.75,14,17666,-50.0,EW,142,382,56.07,Wojciech Wojsa,0.5,13918,True,Wpadka
274598,382,EW,59.75,14,17666,-50.0,EW,142,382,56.07,Michał Szadkowski,0.0,18841,True,Wpadka
274623,382,EW,35.59,27,17666,-100.0,EW,142,382,56.07,Wojciech Wojsa,0.5,13918,True,Wpadka


In [327]:
stat4=tmp[tmp['zapis2']=="Kokosy_na_Obronie"].groupby(['PID'])['wynik'].mean().reset_index(name="Kokosy_na_Obronie")
stat5=tmp[tmp['zapis2']=="częściówka"].groupby(['PID'])['wynik'].mean().reset_index(name="częściówka")
stat6=tmp[tmp['zapis2']=="końcówka"].groupby(['PID'])['wynik'].mean().reset_index(name="końcówka")
stat7=tmp[tmp['zapis2']=="szlemik"].groupby(['PID'])['wynik'].mean().reset_index(name="szlemik")
stat8=tmp[tmp['zapis2']=="Wpadka"].groupby(['PID'])['wynik'].mean().reset_index(name="Wpadka")

In [329]:
stat=stat.merge(stat4,on='PID')
stat=stat.merge(stat5,on='PID')
stat=stat.merge(stat6,on='PID')
stat=stat.merge(stat7,on='PID')
stat=stat.merge(stat8,on='PID')

In [331]:
stat[stat['PID']=='19080']

,PID,średnia_na_rozgrywce,średnia_na_obronie,Kokosy_na_Obronie,częściówka,końcówka,szlemik,Wpadka
1318,19080,58.3558,57.878437,83.805,57.814286,53.679524,72.25125,33.858462


In [332]:
stat.to_csv('statystyki_vol1.csv')

### Count

In [336]:
stats2=tmp[tmp['czy_rozgrywa']==True].groupby(['PID'])['wynik'].count().reset_index(name='liczba_rozdan_na_rozgrywce')
stats3=tmp[tmp['czy_rozgrywa']==False].groupby(['PID'])['wynik'].count().reset_index(name='liczba_rozdan_na_obronie')
stat4=tmp[tmp['zapis2']=="Kokosy_na_Obronie"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_Kokosy_na_Obronie")
stat5=tmp[tmp['zapis2']=="częściówka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_częściówka")
stat6=tmp[tmp['zapis2']=="końcówka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_końcówka")
stat7=tmp[tmp['zapis2']=="szlemik"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_szlemik")
stat8=tmp[tmp['zapis2']=="Wpadka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_Wpadka")

In [337]:
stat=stats2.merge(stats3,on='PID')
stat=stat.merge(stat4,on='PID')
stat=stat.merge(stat5,on='PID')
stat=stat.merge(stat6,on='PID')
stat=stat.merge(stat7,on='PID')
stat=stat.merge(stat8,on='PID')

In [339]:
stats=pd.read_csv('statystyki_vol1.csv')

In [347]:
koniec=pd.concat([stats,stat],axis=1)

In [348]:
koniec.to_csv("Ostateczne.csv")